<a href="https://colab.research.google.com/github/DarthJkid/FBREF_Analysis/blob/main/Top_Five_Leagues_Player_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Data Scraping

In [2]:
df = pd.read_html('https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats', attrs={'id': 'stats_standard'}) [0]
df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                     Rk             Player             Nation   
0                     1         Max Aarons            eng ENG   
1                     2         Max Aarons            eng ENG   
2                     3     Rodrigo Abajas             es ESP   
3                     4     James Abankwah             ie IRL   
4                     5  Keyliane Abdallah             fr FRA   
...                 ...                ...                ...   
2760               2655       Edhy Zuliani             it ITA   
2761               2656   Szymon Żurkowski             pl POL   
2762               2657    Martin Ødegaard             no NOR   
2763               2658        Milan Đurić             ba BIH   
2764               2659        Milan Đurić             ba BIH   

     Unnamed: 3_level_0 Unnamed: 4_level_0  Unnamed: 5_level_0  \
                    Pos              Squad                Comp   
0                    DF        Bournemouth  eng Premier League   
1                    MF           Valencia          es La Liga   
2                    DF           Valencia          es La Liga   
3                 DF,MF            Udinese          it Serie A   
4                    FW          Marseille          fr Ligue 1   
...                 ...                ...                 ...   
2760                 DF           Toulouse          fr Ligue 1   
2761                 MF             Empoli          it Serie A   
2762                 MF            Arsenal  eng Premier League   
2763                 FW              Monza          it Serie A   
2764                 FW              Parma          it Serie A   

     Unnamed: 6_level_0 Unnamed: 7_level_0 Playing Time         ...  \
                    Age               Born           MP Starts  ...   
0                25-040               2000            3      1  ...   
1                25-040               2000            1      0  ...   
2                21-277               2003            1      1  ...   
3                21-028               2004            6      0  ...   
4                18-314               2006            1      0  ...   
...                 ...                ...          ...    ...  ...   
2760             20-186               2004            1      0  ...   
2761             27-141               1997            3      0  ...   
2762             26-058               1998           16     15  ...   
2763             34-267               1990           18     13  ...   
2764             34-267               1990            3      3  ...   

     Per 90 Minutes                                                       \
                Ast   G+A  G-PK G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG   
0              0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
1              0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
2              0.00  0.00  0.00   0.00  0.10  0.00   0.10  0.10     0.10   
3              0.00  0.00  0.00   0.00  0.06  0.00   0.06  0.06     0.06   
4              0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
...             ...   ...   ...    ...   ...   ...    ...   ...      ...   
2760           0.00  0.00  0.00   0.00  0.00  0.00   0.00  0.00     0.00   
2761           0.00  0.00  0.00   0.00  0.10  0.00   0.10  0.10     0.10   
2762           0.21  0.35  0.07   0.28  0.22  0.17   0.39  0.16     0.33   
2763           0.07  0.35  0.28   0.35  0.21  0.08   0.28  0.21     0.28   
2764           0.00  0.00  0.00   0.00  0.12  0.00   0.12  0.12     0.12   

     Unnamed: 37_level_0  
                 Matches  
0                Matches  
1                Matches  
2                Matches  
3                Matches  
4                Matches  
...                  ...  
2760             Matches  
2761             Matches  
2762             Matches  
2763             Matches  
2764             Matches  

[2765 rows x 38 columns]

# Data Cleaning and Preprocessing


In [3]:
#df.columns = ['_'.join(str(c) for c in col).strip() for col in df.columns.values]
#df.head()
# Reset column names to a simple range
df.columns = range(df.shape[1])

# Apply the desired naming logic
df.columns = ['_'.join(str(c) for c in col).strip() if isinstance(col, tuple) else str(col).strip() for col in df.columns]
df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,25-040,2000,3,1,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
1,2,Max Aarons,eng ENG,MF,Valencia,es La Liga,25-040,2000,1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
2,3,Rodrigo Abajas,es ESP,DF,Valencia,es La Liga,21-277,2003,1,1,...,0.00,0.00,0.00,0.00,0.10,0.00,0.10,0.10,0.10,Matches
3,4,James Abankwah,ie IRL,"DF,MF",Udinese,it Serie A,21-028,2004,6,0,...,0.00,0.00,0.00,0.00,0.06,0.00,0.06,0.06,0.06,Matches
4,5,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,18-314,2006,1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
